# Forward Kinematics of the ABB robot using the product of exponentials

We would like to compute the forward kinematics of the ABB [IRB 910SC](https://new.abb.com/products/robotics/robots/scara-robots/irb-910sc) robot seen in class. 

<img src="./figs/ABB_robot.jpg" width="300">

From the [product specification](https://library.e.abb.com/public/9fc170040463477db86bda7065135457/3HAC056431%20PS%20IRB%20910SC-en.pdf?x-sign=Rsw7Ua3yCaT7/Evotwiwx6iTwfZgeP/val93/1o82csyGd1ejrv1gtswH4Bw8ppc) we can gather the following information:
1. The robot has 4 degrees of freedom
<img src="./figs/ABB_dofs.jpg" width="400">
2. The lengths of the different parts are:
<img src="./figs/ABB_lengths.jpg" width="800">

There are three possible versions of the robot with different sizes. We will choose the RB 910SC-3/0.55

From this information we can compute the forward kinematics of the robot. We do the following:
1. Fix a base frame - our world frame of reference
2. Choose frames attached on each link/body (first make sure you find all the links and joints connecting them)
3. Compute the relative transforms between each links
4. Compute the FK giving the pose of the end-effector in the base frame.

Steps 1 and 2 are illustrated on the picture below:
<img src="./figs/ABB_axes.jpg" width="400">

In [1]:
#setup nice plotting
%matplotlib notebook

# we import useful libraries
import time
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

from scipy.linalg import expm

np.set_printoptions(suppress=True, precision=4)

## Forward kinematics from previous notebook

We define below basic functions to get homogeneous transforms that correspond to pure translations and rotations around x,y or z axes. We then use the forward kinematics function we derived in a previous notebook as a reference.

In [2]:
def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:3,3] = vector

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[1:3,1:3] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0,0] = np.cos(angle)
    transform[0,2] = np.sin(angle)
    transform[2,0] = -np.sin(angle)
    transform[2,2] = np.cos(angle)

    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:2,0:2] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    
    ### we return the object
    return transform

def forward_kinematics_ABB(theta):
    # the relative pose of link 1 wrt the base is computed from
    # a translation and then rotation due to the motion of the first joint
    T_SL1 = translate([0,0,0.2577]) @ rotateZ(theta[0])
    
    # the relative pose of link 2 wrt to link 1 is
    # a translation and then rotation due to the motion of the second joint
    T_L1L2 = translate([0.3, 0., 0.]) @ rotateZ(theta[1])
    
    # the relative pose of link 3 wrt to link 2 is
    # a fixed translation followed by a translation due to joint 4 and a rotation due to joint 3
    T_L2L3 = translate([0.25, 0, theta[3]]) @ rotateZ(theta[2])
    
    # finally to get the relative pose from link 3 to the end effector
    # we need to move down and re-orient the frame to match the end-effector frame
    T_L3E = translate([0,0, 0.0402+0.180-.2577])  @ np.array([[0,0,1,0],[0,1,0,0],[-1,0,0,0],[0,0,0,1]])
    
    # the forward kinematics is then
    T_SF = T_SL1 @ T_L1L2 @ T_L2L3 @ T_L3E
    
    ### we return the pose of the end-effector
    return T_SF

## Forward kinematics using the product of exponentials

Below we derive the forward kinematics using the product of exponentials. First we implement two simple helper functions to compute the bracket of a 3D vector (i.e. $[\omega]$) with the function `to_skew` and then the bracket of a twist ($[V] = \begin{bmatrix} [\omega] & v \\ 0 & 0 \end{bmatrix}$) with the function `to_bracket`.

Consider the robot at its 0 position (i.e. all its joints are at 0). We compute $M$ the pose of the end-effector at the 0 position. Looking at the datasheet and our convention for the end-effector frame, we easily find that 
$$M = \begin{bmatrix} 0 & 0 & 1 & 0.55 \\ 0 & 1 & 0 & 0 \\ -1 & 0 & 0 & 0.2202 \\ 0 & 0 & 0 & 1 \end{bmatrix}$$

We then compute the screw axes of each joint in the S frame
1. For $S_1$ the rotation axis is aligned with the z axis of the S frame and the S frame's origin is on the rotation axes (so there is no linear velocity from S) when then have
$$S_1 = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \end{pmatrix}$$

2. For $S_2$ the rotation axis is aligned with the z axis of the S frame so the top part if $\omega_2^S = (0,0,1)$. The position of joint 2 with respect to S is $p_{S2} = (0.3, 0, 0.2577)$ and therefore $v_2^S = \omega_2^S \times p_{2S} = -\omega_2^S \times p_{S2} = (0,-0.3,0)$ so we get
$$S_2 = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \\ -0.3 \\ 0 \end{pmatrix}$$

3. A similar computation for $S_3$ gives
$$S_3 = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \\ -0.55 \\ 0 \end{pmatrix}$$

4. The last joint is a translation so the screw is also a pure translation in the S frame which is aligned with the  z axis so we have
$$S_4 = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{pmatrix}$$

The forward kinematics is then computed as
$$FK(\theta_1, \theta_2, \theta_3, \theta_4) = \mathrm{e}^{[S_1] \theta_1}\cdot\mathrm{e}^{[S_2] \theta_2}\cdot \mathrm{e}^{[S_3] \theta_3} \cdot \mathrm{e}^{[S_4] \theta_4} \cdot M$$

The complete code is given below

In [3]:
def to_skew(w):
    om_skew = np.zeros([3,3])
    om_skew[0,1] = -w[2]
    om_skew[1,0] = w[2]
    om_skew[0,2] = w[1]
    om_skew[2,0] = -w[1]
    om_skew[1,2] = -w[0]
    om_skew[2,1] = w[0]
    return om_skew

def to_bracket(V):
    v_brack = np.zeros([4,4])
    v_brack[0:3, 0:3] = to_skew(V[0:3])
    v_brack[0:3,3] = V[3:6]
    return v_brack

def forward_kinematics_ABB_exponential(theta):
    S1 = to_bracket([0,0,1,0,0,0])
    S2 = to_bracket([0,0,1,0,-0.3,0])
    S3 = to_bracket([0,0,1,0,-0.55,0])
    S4 = to_bracket([0,0,0,0,0,1])
    M = np.eye(4)
    M[:,:] = np.array([[0,0,1,0],[0,1,0,0],[-1,0,0,0],[0,0,0,1]])
    M[0:3,3] = np.array([0.55,0,0.2577 + 0.0402+0.180-.2577])
        
    return expm(theta[0]*S1) @ expm(theta[1]*S2) @ expm(theta[2]*S3) @ expm(theta[3]*S4) @ M

In [4]:
# now we can compute the pose of the end-effector for various values of the joints
theta = np.array([0,0,0,0.])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\n Using our old FK function, we find the pose of the end effector to be\n\n {T}\n\n\n')

print(f'We also verify that with the product of exponential formula we get\n\n {forward_kinematics_ABB_exponential(theta)}\n\n\n')

for theta = [0. 0. 0. 0.]

 Using our old FK function, we find the pose of the end effector to be

 [[ 0.      0.      1.      0.55  ]
 [ 0.      1.      0.      0.    ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]



We also verify that with the product of exponential formula we get

 [[ 0.      0.      1.      0.55  ]
 [ 0.      1.      0.      0.    ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]





In [5]:
theta = np.array([np.pi/2,0,0,0.])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\n Using our old FK function, we find the pose of the end effector to be\n\n {T}\n\n\n')

print(f'We also verify that with the product of exponential formula we get\n\n {forward_kinematics_ABB_exponential(theta)}\n\n\n')

for theta = [1.5708 0.     0.     0.    ]

 Using our old FK function, we find the pose of the end effector to be

 [[ 0.     -1.      0.      0.    ]
 [ 0.      0.      1.      0.55  ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]



We also verify that with the product of exponential formula we get

 [[ 0.     -1.      0.      0.    ]
 [ 0.      0.      1.      0.55  ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]





In [6]:
# now we generate random values for theta
theta = np.random.rand(4)
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\n Using our old FK function, we find the pose of the end effector to be\n\n {T}\n\n\n')

print(f'We also verify that with the product of exponential formula we get\n\n {forward_kinematics_ABB_exponential(theta)}\n\n\n')

for theta = [0.5035 0.31   0.6979 0.5463]

 Using our old FK function, we find the pose of the end effector to be

 [[ 0.     -0.9982  0.0594  0.4345]
 [ 0.      0.0594  0.9982  0.3264]
 [-1.      0.      0.      0.7665]
 [ 0.      0.      0.      1.    ]]



We also verify that with the product of exponential formula we get

 [[ 0.     -0.9982  0.0594  0.4345]
 [ 0.      0.0594  0.9982  0.3264]
 [-1.      0.      0.      0.7665]
 [ 0.      0.      0.      1.    ]]



